In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('clean3.csv')
df_copy = df.copy()
df_copy.head(3)

,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,ApprovalDate,Term,NoEmp,...,ApprovalDate_day,Population,PopDensity,HousingUnits,OccupiedHousingUnits,MedianHomeValue,MedianHouseholdIncome,USDPerJobCreated,DisbursePerEmployee,GrSBA_AppvDiff%
0,1000014003,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,1997-02-28,84.0,4.0,...,28.0,31457.0,1939.0,14267.0,12895.0,115600.0,48618.0,60000.0,12000.000000,20.0
1,1000024006,LANDMARK BAR & GRILLE (THE),NEW PARIS,IN,46526,1ST SOURCE BANK,IN,1997-02-28,60.0,2.0,...,28.0,30671.0,457.0,11847.0,10938.0,120200.0,47863.0,40000.0,13333.333333,20.0
2,1000034009,"WHITLOCK DDS, TODD M.",BLOOMINGTON,IN,47401,GRANT COUNTY STATE BANK,IN,1997-02-28,180.0,7.0,...,28.0,41011.0,495.0,19124.0,17562.0,207900.0,45415.0,287000.0,35875.000000,25.0
